# 타이타닉 생존자 예측

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
df = sns.load_dataset('titanic')
df.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True


### 1. 데이터 전처리

- Feature selection

In [3]:
df = df[['survived','pclass','sex','age','sibsp','parch','fare','embarked','deck']]
df.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,deck
0,0,3,male,22.0,1,0,7.2500,S,NaN
1,1,1,female,38.0,1,0,71.2833,C,C
2,1,3,female,26.0,0,0,7.9250,S,NaN


- 결측치 처리

In [4]:
# 결측치 확인
df.isna().sum()

survived      0
pclass        0
sex           0
age         177
sibsp         0
parch         0
fare          0
embarked      2
deck        688
dtype: int64

In [5]:
# age 컬럼은 평균으로 대체
df.age.fillna(df.age.mean(), inplace=True)
df.age.isna().sum()

0

In [6]:
# embarked 컬럼은 최빈값으로 대체
df.embarked.value_counts()

S    644
C    168
Q     77
Name: embarked, dtype: int64

In [7]:
df.embarked.fillna('S', inplace=True)
df.embarked.isna().sum()

0

In [8]:
# deck 컬럼은 삭제
df.drop(columns=['deck'], inplace=True)
df.isna().sum().sum()

0

- 카테고리 값인 sex, embarked 컬럼은 숫자로 변환

In [9]:
# LabelEncoder로 변환
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [10]:
df.sex = le.fit_transform(df.sex)
df.embarked = le.fit_transform(df.embarked)
df.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2


### 2. Train/Test dataset으로 분리

In [11]:
# X와 y를 넘파이 배열로
X = df.iloc[:,1:].values
y = df.survived.values
X.shape, y.shape

((891, 7), (891,))

In [12]:
# y값의 분포
df.survived.value_counts()

0    549
1    342
Name: survived, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=2021
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 7), (179, 7), (712,), (179,))

In [14]:
np.unique(y_train, return_counts=True)

(array([0, 1], dtype=int64), array([439, 273], dtype=int64))

### 3. RandomForest 모델로 학습

In [16]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=2021)
rfc.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 2021,
 'verbose': 0,
 'warm_start': False}

In [17]:
# RFC로 학습
rfc.fit(X_train, y_train)

RandomForestClassifier(random_state=2021)

### 4. 모델 예측 및 평가

In [18]:
rfc.score(X_test, y_test)

0.8100558659217877

### 5. 3, 4 대신에 GridSearchCV 수행

In [19]:
params = {
    'max_depth': [2,4,6,8],
    'min_samples_split': [2,4,6]
}

In [20]:
from sklearn.model_selection import GridSearchCV
grid_rf = GridSearchCV(rfc, param_grid=params, scoring='accuracy', cv=5)
grid_rf.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=2021),
             param_grid={'max_depth': [2, 4, 6, 8],
                         'min_samples_split': [2, 4, 6]},
             scoring='accuracy')

In [21]:
grid_rf.best_params_

{'max_depth': 8, 'min_samples_split': 4}

In [22]:
params = {
    'max_depth': [7,8,9,10],
    'min_samples_split': [3,4,5]
}
grid_rf = GridSearchCV(rfc, param_grid=params, scoring='accuracy', cv=5)
grid_rf.fit(X_train, y_train)
grid_rf.best_params_

{'max_depth': 8, 'min_samples_split': 4}

In [23]:
best_rf = grid_rf.best_estimator_
best_rf.score(X_test, y_test)

0.8100558659217877

### 6. 테스트 데이터에 적용

In [24]:
X_test[27], y_test[27]

(array([ 3.    ,  1.    , 32.    ,  0.    ,  0.    , 56.4958,  2.    ]), 1)

In [26]:
grid_rf.predict(X_test[27].reshape(1,-1))

array([0], dtype=int64)

In [27]:
grid_rf.predict(X_test[12].reshape(1,-1)), y_test[12]

(array([0], dtype=int64), 0)

In [29]:
X_test[12]

array([ 2. ,  1. , 16. ,  0. ,  0. , 10.5,  2. ])

### 7. 엉터리 분류기
- 여성이면 생존이라 예측, 그외는 사망

In [31]:
df.groupby('sex')['survived'].mean()

sex
0    0.742038
1    0.188908
Name: survived, dtype: float64

In [33]:
df.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2


In [35]:
X_test.shape, y_test.shape

((179, 7), (179,))

In [36]:
from sklearn.base import BaseEstimator

class MyClassifier(BaseEstimator):
    # fit(), predict() method만 재정의(override)
    def fit(self, X, y):
        pass
    def predict(self, X):
        pred = np.zeros(X.shape[0])
        for i in range(X.shape[0]):
            if X[i, 1] == 0:        # 여성이면 생존으로 처리
                pred[i] = 1
        ''' pred = np.empty(X.shape[0])
        for i in range(X.shape[0]):
            pred[i] = 1 if X[i,0] == 0 else 0 '''
        return pred

In [37]:
my_clf = MyClassifier()
my_clf.fit(X_train, y_train)
pred_my = my_clf.predict(X_test)

In [41]:
X_test[:5, 1], pred_my[:5]

(array([0., 1., 0., 1., 0.]), array([1., 0., 1., 0., 1.]))

In [42]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred_my)

0.7877094972067039

In [43]:
pred_rf = best_rf.predict(X_test)
accuracy_score(y_test, pred_rf)

0.8100558659217877

In [44]:
sdf = pd.DataFrame({'y_test':y_test, 'RF':pred_rf, 'My':pred_my})
sdf.head()

,y_test,RF,My
0,1,0,1.0
1,1,0,0.0
2,1,1,1.0
3,0,0,0.0
4,0,1,1.0


- 모델의 성능을 평가할 때 무조건적으로 정확도를 사용하는 것은 지양

In [45]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score
confusion_matrix(y_test, pred_rf)

array([[98, 12],
       [22, 47]], dtype=int64)

In [46]:
confusion_matrix(y_test, pred_my)

array([[95, 15],
       [23, 46]], dtype=int64)

In [47]:
precision_score(y_test, pred_rf), recall_score(y_test, pred_rf)

(0.7966101694915254, 0.6811594202898551)

In [48]:
precision_score(y_test, pred_my), recall_score(y_test, pred_my)

(0.7540983606557377, 0.6666666666666666)